In [44]:
import requests
import pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from shapely import wkt
from shapely.geometry import Polygon

In [45]:
# Function to extract geometry information from json and save it in Pandas DataFrame
def json_to_dataframe(json_data):
    # Extract the features from the JSON data
    features = json_data['features']
    
    # Extract the geometry information
    data = [feature['geometry'] for feature in features]
    
    # Create a DataFrame from the data
    df = pd.DataFrame(data)
    
    # Extract the polygon from the data
    df['geometry'] = df.iloc[:, 0].apply(lambda x: x[0])
    
    return df

In [46]:
# Function to calculate the centroid of polygons
def get_centroid(coords):
    polygon = Polygon(coords)
    return polygon.centroid

In [47]:
# Function to get the geometry information of the parcel data from ArcGIS service
def get_parcel_data(state, bpoly):
    # Get the URL of the state parcel data
    df = pd.read_csv('us_states_parcel_data.csv') # We might want to change the directory to the spreadsheet
    url = df.loc[df['State'] == state, 'URL'].values[0]
    
    # Get the coordinates of the bounding box for input polygon bpoly
    bbox = bpoly.bounds
    
    # Construct the query URL to get the parcel data within the bounding box
    query = (url + '/0/query?' +
             f'geometry={bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}' +
             '&geometryType=esriGeometryEnvelope&inSR=4326' + 
             '&spatialRel=esriSpatialRelIntersects' +
             '&outFields=*' +
             '&returnGeometry=true&f=json')
    
    # Set up a session with retries
    s = requests.Session()
    retries = Retry(total=5, 
                    backoff_factor=0.1,
                    status_forcelist=[500, 502, 503, 504])
    s.mount('https://', HTTPAdapter(max_retries=retries))

    # Send the request and get the response
    r = s.get(query)
    
    # Get the parcel data in JSON format
    parcel_data = r.json()
    
    # Extract the geometry information from the JSON file and store in the csv format
    df = json_to_dataframe(parcel_data)

    # Calculate the centroid for each parcel
    df['centroid'] = df['geometry'].apply(lambda x: get_centroid(x))

    # Extract x (longitude) and y (latitude) coordinates
    df['xcoord'] = df['centroid'].apply(lambda point: point.x)
    df['ycoord'] = df['centroid'].apply(lambda point: point.y)
    
    return df

# Alameda Island Example

In [49]:
### CHANGE HERE (Alameda Island Example)
from shapely.geometry import box, shape
# Define the bounding box for Alameda Island
bbox = box(-122.3164, 37.7393, -122.2307, 37.7957)

# Get the parcel data for Alameda Island in JSON format
parcel_data = get_parcel_data('California', bbox)

parcel_data.head()

,rings,geometry,centroid,xcoord,ycoord
0,"[[[-122.253860704, 37.7448346140001], [-122.25...","[[-122.253860704, 37.7448346140001], [-122.253...",POINT (-122.25372373679335 37.74491824741586),-122.253724,37.744918
1,"[[[-122.254360611, 37.7422976560001], [-122.25...","[[-122.254360611, 37.7422976560001], [-122.254...",POINT (-122.25434766088618 37.742450380181104),-122.254348,37.742450
2,"[[[-122.259890339, 37.7427485740001], [-122.26...","[[-122.259890339, 37.7427485740001], [-122.260...",POINT (-122.25993242428014 37.74290950109544),-122.259932,37.742910
3,"[[[-122.246943335, 37.741273151], [-122.247113...","[[-122.246943335, 37.741273151], [-122.2471134...",POINT (-122.24706183558392 37.741359644284636),-122.247062,37.741360
4,"[[[-122.241465615, 37.7466822150001], [-122.24...","[[-122.241465615, 37.7466822150001], [-122.241...",POINT (-122.24133563024462 37.746767918888935),-122.241336,37.746768


In [ ]:
parcel_data.to_csv('parcel.csv')